In [1]:
import tensorflow as tf
import pandas as pd

In [2]:
import numpy as np

In [3]:
tf.compat.v1.enable_eager_execution()
# Define the feature description for the serialized Example protocol buffer
feature_description = {
    'tensor_output': tf.io.FixedLenFeature([], tf.string)
}

# Create a dataset from the TFRecord file
dataset = tf.data.TFRecordDataset('message_gns_gpu.tfrecord')

# Parse the serialized Example protocol buffer messages to extract the tensor values
def parse_example(example_proto):
    example = tf.io.parse_single_example(example_proto, feature_description)
    tensor_proto = tf.io.parse_tensor(example['tensor_output'], out_type=tf.float32)
    return tensor_proto

# Map the parse function to the dataset
parsed_dataset = dataset.map(parse_example)
message = []
# Iterate through the dataset and print the tensor values
for tensor_value in parsed_dataset:
        message.append(tensor_value)

message = tf.stack(message)

In [4]:
print(message.shape)

(2000, 336, 3, 128)


In [4]:
s1=message[:,:,0,:3]
s2=message[:,:,1,:]
m12=message[:,:,2,:]
print(s1.shape)
print(s2.shape)
print(m12.shape)

(2000, 336, 3)
(2000, 336, 128)
(2000, 336, 128)


In [43]:
new_mess = tf.concat([s1, m12], axis=-1)
print(new_mess.shape)
new_message = tf.reshape(new_mess, (672000,131))
print(new_message.shape)

(2000, 336, 131)
(672000, 131)


In [5]:
m12_2d = tf.reshape(m12, (672000,128))

In [6]:
s1_2d = tf.reshape(s1, (672000,3))
s1_2d.shape

TensorShape([Dimension(672000), Dimension(3)])

In [7]:
column_s1 = [elem%(k) for k in range(1, 3) for elem in 'x%d y%d z%d'.split(' ')]

In [16]:
# assuming s1 is your original tensor with shape (2000, 336, 3)
s1_2d = tf.reshape(s1, (672000, 3))

# create a list of column names with 3 elements
column_s1 = ['dx', 'dy', 'dz']

# create a pandas DataFrame with the reshaped tensor and column names
a = pd.DataFrame(data=s1_2d.cpu().numpy(), columns=column_s1)

In [19]:
a['r'] = (a['dx'])**2 + (a['dy'])**2 + (a['dz'])**2

In [21]:
# columns = [elem%(k) for k in range(1, 3) for elem in 'x%d y%d z%d vx%d vy%d vz%d'.split(' ')]
# columns = [elem%(k) for k in range(1, 3) for elem in 'x%d y%d z%d'.split(' ')]
# columns += ['e%d'%(k,) for k in range(128)]
columns = ['e%d'%(k,) for k in range(128)]

In [22]:
b = pd.DataFrame(data=m12_2d[:,:].cpu().numpy(),
            columns=columns)

In [23]:
joined_df = pd.concat([a, b], axis=1)

In [24]:
joined_df

,dx,dy,dz,r,e0,e1,e2,e3,e4,e5,...,e118,e119,e120,e121,e122,e123,e124,e125,e126,e127
0,1.278152,0.881601,0.602907,2.774390,3.677854,-0.324887,0.068612,3.255036,0.583415,-2.199551,...,-0.686014,-0.201834,3.126923,-0.461876,-1.973737,0.427768,-1.182460,1.058521,-1.829934,0.673095
1,1.278152,0.881601,0.602907,2.774390,3.677854,-0.324887,0.068612,3.255036,0.583415,-2.199551,...,-0.686014,-0.201834,3.126923,-0.461876,-1.973737,0.427768,-1.182460,1.058521,-1.829934,0.673095
2,1.278152,0.881601,0.602907,2.774390,3.677854,-0.324887,0.068612,3.255036,0.583415,-2.199551,...,-0.686014,-0.201834,3.126923,-0.461876,-1.973737,0.427768,-1.182460,1.058521,-1.829934,0.673095
3,1.278152,0.881601,0.602907,2.774390,3.677854,-0.324887,0.068612,3.255036,0.583415,-2.199551,...,-0.686014,-0.201834,3.126923,-0.461876,-1.973737,0.427768,-1.182460,1.058521,-1.829934,0.673095
4,1.278152,0.881601,0.602907,2.774390,3.677854,-0.324887,0.068612,3.255036,0.583415,-2.199551,...,-0.686014,-0.201834,3.126923,-0.461876,-1.973737,0.427768,-1.182460,1.058521,-1.829934,0.673095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
671995,-0.056598,-0.119295,-0.192969,0.054672,0.247286,-0.014542,-0.000372,-0.222942,-0.000131,-0.640224,...,-0.000800,-0.243040,0.077805,0.225139,-1.123919,0.000045,-0.439335,-0.320556,0.014990,-0.340962
671996,-0.056598,-0.119295,-0.192969,0.054672,0.247286,-0.014542,-0.000372,-0.222942,-0.000131,-0.640224,...,-0.000800,-0.243040,0.077805,0.225139,-1.123919,0.000045,-0.439335,-0.320556,0.014990,-0.340962
671997,-0.056598,-0.119295,-0.192969,0.054672,0.247286,-0.014542,-0.000372,-0.222942,-0.000131,-0.640224,...,-0.000800,-0.243040,0.077805,0.225139,-1.123919,0.000045,-0.439335,-0.320556,0.014990,-0.340962
671998,-0.056598,-0.119295,-0.192969,0.054672,0.247286,-0.014542,-0.000372,-0.222942,-0.000131,-0.640224,...,-0.000800,-0.243040,0.077805,0.225139,-1.123919,0.000045,-0.439335,-0.320556,0.014990,-0.340962


In [35]:
best_message = np.argmax([np.std(joined_df[:-1]['e%d'%(i,)]) for i in range(128)])

In [39]:
best = joined_df[:-1][['e%d'%(best_message,), 'dx', 'dy', 'dz', 'r']]

In [40]:
best

,e98,dx,dy,dz,r
0,1.000440,1.278152,0.881601,0.602907,2.774390
1,1.000440,1.278152,0.881601,0.602907,2.774390
2,1.000440,1.278152,0.881601,0.602907,2.774390
3,1.000440,1.278152,0.881601,0.602907,2.774390
4,1.000440,1.278152,0.881601,0.602907,2.774390
...,...,...,...,...,...
671994,-2.036776,-0.056598,-0.119295,-0.192969,0.054672
671995,-2.036776,-0.056598,-0.119295,-0.192969,0.054672
671996,-2.036776,-0.056598,-0.119295,-0.192969,0.054672
671997,-2.036776,-0.056598,-0.119295,-0.192969,0.054672


In [41]:
import pickle as pkl
pkl.dump(best,
    open('best_message_nri.pkl', 'wb'))